In [33]:
# Authentification and initialization of the project

import ee
import os

# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize(project='ee-willmaymbenza')

Enter verification code: 4/1AdLIrYf9j5OZu4oyGUTb5PfaZ5iNVeM2G7QpQ2kl5nKucuEaA3B0FuDesf0

Successfully saved authorization token.


In [36]:
pip install geemap

Note: you may need to restart the kernel to use updated packages.


In [37]:
pip install eemont

Note: you may need to restart the kernel to use updated packages.


In [38]:
!pip install pycrs

In [39]:
pip install geopandas

Note: you may need to restart the kernel to use updated packages.


In [40]:
# importing relevant packages for the project

import geemap
import geemap.colormaps as cm
import geopandas as gpd
import eemont

In [41]:
# Area of interest and reference geometric point

point = ee.Geometry.Point(136.99805556, 35.09138889) #Nagoya, Japan
#Keep in mind that Earth Engine uses the convention (Lon E/O, Lat N/S)

aoi = point.buffer(10000).bounds() # 10km buffer around the point

# Load the shapefile
roi = '/Users/suimon/Documents/Master course/Research/Flood Mitigation/Catchment_definition/Vectorized_upslope.shp'

# Define the CRS parameters for EPSG:32653
crs_params = {
    'name': 'EPSG32653',
    'custom': False
}

# Create a Map object with EPSG 32653 projection
Map = geemap.Map()
Map.add_basemap ('HYBRID', False)
# Center the map on the region of interest
#Map.centerObject(roi, 12)

# Load the shapefile as a feature collection
watershed = geemap.shp_to_ee(roi)


In [42]:
# Date range

start_l9 = '2023-01-01'
end_l9 = '2023-12-31'

dataset_l9 = (ee.ImageCollection('LANDSAT/LC09/C02/T1_L2').filterDate(start_l9, end_l9)
        .filterBounds(watershed)
        .filter((ee.Filter.lt('CLOUD_COVER', 10))))


# Applies scaling factors.
def apply_scale_factors(image):
  optical_bands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
  thermal_bands = image.select('ST_B.*').multiply(0.00341802).add(149.0)
  return image.addBands(optical_bands, None, True).addBands(
      thermal_bands, None, True
  )

# Rescaling the dataset

rescale = dataset_l9.map(apply_scale_factors)
l9 = rescale.median()

visualization = {
    'bands': ['SR_B4', 'SR_B3', 'SR_B2'],
    'min': 0.0,
    'max': 0.3,
}

Map.set_center(136.99805556, 35.09138889, 8)
Map.add_layer(l9, visualization, 'Landsat 9')
Map.addLayer(watershed, {}, 'ROI')
Map

Map(center=[35.09138889, 136.99805556], controls=(WidgetControl(options=['position', 'transparent_bg'], widget…

In [43]:
# creating training data

load_samples = gpd.read_file('/Users/suimon/Downloads/Training_points-1.csv')
#print(load_samples.head())
combined_samples = load_samples
combined_samples

,system:index,Class,.geo,geometry
0,1_1_1_0,0,"{""type"":""Point"",""coordinates"":[136.70525490994...",None
1,1_1_1_1,0,"{""type"":""Point"",""coordinates"":[136.70525490994...",None
2,1_1_1_2,0,"{""type"":""Point"",""coordinates"":[136.71486794705...",None
3,1_1_1_3,0,"{""type"":""Point"",""coordinates"":[136.71486794705...",None
4,1_1_1_4,0,"{""type"":""Point"",""coordinates"":[136.71486794705...",None
...,...,...,...,...
3995,2_995,3,"{""type"":""Point"",""coordinates"":[136.96582968619...",None
3996,2_996,3,"{""type"":""Point"",""coordinates"":[136.96602816966...",None
3997,2_997,3,"{""type"":""Point"",""coordinates"":[136.96592624572...",None
3998,2_998,3,"{""type"":""Point"",""coordinates"":[136.96564193156...",None


In [44]:
print (combined_samples.head())

  system:index Class                                               .geo  \
0      1_1_1_0     0  {"type":"Point","coordinates":[136.70525490994...   
1      1_1_1_1     0  {"type":"Point","coordinates":[136.70525490994...   
2      1_1_1_2     0  {"type":"Point","coordinates":[136.71486794705...   
3      1_1_1_3     0  {"type":"Point","coordinates":[136.71486794705...   
4      1_1_1_4     0  {"type":"Point","coordinates":[136.71486794705...   

  geometry  
0     None  
1     None  
2     None  
3     None  
4     None  


In [75]:
# Load sample vectors as FeatureCollection (e.g., points or polygons)
samples = ee.FeatureCollection('projects/ee-willmaymbenza/assets/Training_points')

sampled_landsat = l9.sampleRegions(collection=samples,
                                            properties=['Class'],
                                            scale=30,
                                            projection = l9.projection(),
                                            geometries = True)

In [46]:
samples

In [70]:
print(training.first().getInfo())

{'type': 'Feature', 'geometry': None, 'id': '00000000000000000000_0', 'properties': {'SR_B1': 0.0009149999999999853, 'SR_B2': 0.011433749999999993, 'SR_B3': 0.02254375, 'SR_B4': 0.007624999999999993, 'SR_B5': 0.0037750000000000006, 'SR_B7': 0.0099075}}


In [76]:
sampled_landsat.first().getInfo()

{'type': 'Feature',
 'geometry': {'geodesic': False,
  'type': 'Point',
  'coordinates': [136.70530552695936, 34.97972342715108]},
 'id': '00000000000000000000_0',
 'properties': {'Class': 0,
  'QA_PIXEL': 21952,
  'QA_RADSAT': 0,
  'SR_B1': 0.0009149999999999853,
  'SR_B2': 0.011433749999999993,
  'SR_B3': 0.02254375,
  'SR_B4': 0.007624999999999993,
  'SR_B5': 0.0037750000000000006,
  'SR_B6': 0.010237499999999997,
  'SR_B7': 0.0099075,
  'SR_QA_AEROSOL': 208,
  'ST_ATRAN': 8872.5,
  'ST_B10': 289.29263089999995,
  'ST_CDIST': 154,
  'ST_DRAD': 396,
  'ST_EMIS': 9880,
  'ST_EMSD': 0,
  'ST_QA': 304.5,
  'ST_TRAD': 7925,
  'ST_URAD': 755.5}}

In [86]:
# Bands for prediction

label = 'Class'
bands = ['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7']
input_data = l9.select(bands)

# Overlay the points on the imagery to get training

trainImage = input_data.sampleRegions(**{
    'collection': sampled_landsat,
    'properties': [label],
    'scale': 30
})

In [87]:
# split the samples into training and testing sets
training_data = trainImage.randomColumn()

train_set = training_data.filter(ee.Filter.lessThan('random', 0.8))
test_set = training_data.filter(ee.Filter.greaterThanOrEquals('random', 0.8))

#Train a Random Forest classifier

classifier = ee.Classifier.smileRandomForest(15).train(
    features = train_set, 
    classProperty = 'Class', 
    inputProperties = bands)

# classify the image
classified = input_data.classify(classifier)
clip_class_l9 = classified.clip(watershed)


In [90]:
# Defining a palette for the classification

landcoverPalette = [
  '#0000ff', # water (0)
  '#ff0000', # urban (1)
  '#fffd00', # barren(2)
  '#00ff02', # vegetation (3)
  ]

# Visualizing the classification

Map.addLayer(clip_class_l9, {'palette': ['blue', 'red', 'yellow', 'green'], 'min': 0,  'max':3}, 'Classification L9')


In [91]:
print (classified.getInfo())

{'type': 'Image', 'bands': [{'id': 'classification', 'data_type': {'type': 'PixelType', 'precision': 'int', 'min': -2147483648, 'max': 2147483647}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}]}


In [96]:
# Accuracy assessment

# Classify the test set
classified_test_set = test_set.classify(classifier)

# Create the confusion matrix
confusionMatrix = ee.ConfusionMatrix(classified_test_set.errorMatrix('Class', 'classification'))

# Print the confusion matrix
print('Confusion matrix :', confusionMatrix.getInfo())

# Print the overall accuracy
print('Overall Accuracy :', confusionMatrix.accuracy().getInfo())

# Print the Kappa coefficient
print('Kappa test :', confusionMatrix.kappa().getInfo())

# Print the producer's accuracy
print('Producer\'s Accuracy :', confusionMatrix.producersAccuracy().getInfo())

# Print the consumer's accuracy
print('Consumer\'s Accuracy :', confusionMatrix.consumersAccuracy().getInfo())


Confusion matrix : [[199, 5, 0, 5], [5, 187, 1, 2], [0, 3, 201, 1], [1, 0, 0, 208]]
Overall Accuracy : 0.9718826405867971
Kappa test : 0.9624995016942396
Producer's Accuracy : [[0.9521531100478469], [0.958974358974359], [0.9804878048780488], [0.9952153110047847]]
Consumer's Accuracy : [[0.9707317073170731, 0.958974358974359, 0.995049504950495, 0.9629629629629629]]


In [102]:
# Overall change detection 

areaImage = ee.Image.pixelArea().addBands(clip_class_l9)

areas_l9 = areaImage.reduceRegion(
  reducer = ee.Reducer.sum().group(
    groupField= 1,
    groupName= 'Class'),
    geometry= watershed,
    scale= 30
)

print(areas_l9.getInfo())

{'groups': [{'Class': 0, 'sum': 4858959.427611108}, {'Class': 1, 'sum': 40692213.397805326}, {'Class': 2, 'sum': 3965779.2642527865}, {'Class': 3, 'sum': 5899842.772711299}]}


In [105]:
import os

# Get the current working directory
current_directory = os.getcwd()

print(f'The current working directory is: {current_directory}')



The current working directory is: /Users/suimon/Python
